In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
from transformers import BertModel
import torch.nn as nn
from transformers import BertConfig, BertTokenizer
import numpy as np
from transformers import AdamW
from tqdm.auto import tqdm
import json
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 3.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.3 MB/s 
     |████████████████████████████████| 120 kB 43.8 MB/s 


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
min_len, max_len = 128, 256
N = 3

In [4]:
gdrive_path = '/content/gdrive/MyDrive/PQUAD_DATA/'
train_path = gdrive_path + 'Train_v8.0.json'
test_path = gdrive_path + 'test2.json'
validation_path = gdrive_path + 'validation2.json'

In [5]:
def json_to_dataframe(file):
    f = open ( file , "r") 
    data = json.loads(f.read())               #loading the json file.
    iid = []                                  
    tit = []                                  #Creating empty lists to store values.
    con = []
    Que = []
    Ans_st = []
    Txt = []
    
    for i in range(len(data['data'])):       #Root tag of the json file contains 'title' tag & 'paragraphs' list.
        title = data['data'][i]['title']
        for p in range(len(data['data'][i]['paragraphs'])):  # 'paragraphs' list contains 'context' tag & 'qas' list.
            context = data['data'][i]['paragraphs'][p]['context']
            for q in range(len(data['data'][i]['paragraphs'][p]['qas'])):  # 'qas' list contains 'question', 'Id' tag & 'answers' list.
                question = data['data'][i]['paragraphs'][p]['qas'][q]['question']
                Id = data['data'][i]['paragraphs'][p]['qas'][q]['id']
                if len(data['data'][i]['paragraphs'][p]['qas'][q]['answers']) == 0 :
                  ans_start = -1
                  text = ''
                  tit.append(title)
                  con.append(context)
                  Que.append(question)                    # Appending values to lists
                  iid.append(Id)
                  Ans_st.append(ans_start)
                  Txt.append(text)
                else :   
                  for a in range(len(data['data'][i]['paragraphs'][p]['qas'][q]['answers'])): # 'answers' list contains 'ans_start', 'text' tags. 
                      ans_start = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start']
                      text = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['text']
                                          
                      tit.append(title)
                      con.append(context)
                      Que.append(question)                    # Appending values to lists
                      iid.append(Id)
                      Ans_st.append(ans_start)
                      Txt.append(text)

    new_df = pd.DataFrame(columns=['Id','title','context','question','ans_start','text']) # Creating empty DataFrame.
    new_df.Id = iid
    new_df.title = tit           #intializing list values to the DataFrame.
    new_df.context = con
    new_df.question = Que
    new_df.ans_start = Ans_st
    new_df.text = Txt

    final_df = new_df.drop_duplicates(keep='first')  # Dropping duplicate rows from the create Dataframe.
    return final_df


df_train = json_to_dataframe(train_path)   
train_rows = df_train.shape[0] 
print('size of Train dataframe before concatenation is {}'.format(train_rows))

df_test = json_to_dataframe(test_path)  
df_validation = json_to_dataframe(validation_path)   


frames = [df_train, df_validation]
df_train = pd.concat(frames)
train_rows = df_train.shape[0] 
print('size of Train dataframe after concatenation is {}'.format(train_rows))
df_train.head()

size of Train dataframe before concatenation is 63998
size of Train dataframe after concatenation is 74383


,Id,title,context,question,ans_start,text
0,101001,آرسنال,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,موقعیت جغرافی باشگاه فوتبال آرسنال را بگویید؟,86,شمال شهر لندن
1,101002,آرسنال,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,لیگ برتر انگلستان موفق به کسب چند عنوان قهرمان...,173,۱۴
2,101003,آرسنال,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,بیشترین بازی بدون باخت پیاپی متعلق به کدام باش...,61,باشگاه فوتبال انگلیسی
3,101004,آرسنال,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,باشگاه فوتبال آرسنال موفق به کسب چند عنوان قهر...,119,۱۳
4,101005,آرسنال,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,باشگاه فوتبال آرسنال چند عنوان قهرمانی در جام ...,214,۱۶


In [6]:
def add_end_idx(answers_text, answers_start, contexts):
    new_answers = []
    # loop through each answer-context pair
    for answer_text, answer_start, context in tqdm(zip(answers_text, answers_start, contexts)):
        
        start_shift = 0
        text = re.sub("^\u200c", "", answer_text) #remove start half-spaces
        if len(list(text)) == (len(list(answer_text)) - 1):
          start_shift += 1
        text = re.sub("\u200c$", "", text) #remove end half-spaces
        
        text = re.sub("^\s+",'',text)
        if len(list(text)) == (len(list(answer_text)) - 1):
          start_shift += 1
        text = re.sub("\s+$",'',text)

        answer_start += start_shift
        end_idx = answer_start + len(text)
        # ...however, sometimes squad answers are off by a character or two
        if context[answer_start:end_idx] == text:
            # if the answer is not off :)
            answer_end = end_idx
        else:
            # this means the answer is off by 1-2 tokens
            for n in [1, 2]:
                if context[answer_start-n:end_idx-n] == text:
                    answer_start = answer_start - n
                    answer_end = end_idx - n
        new_answers.append({'text': text,'answer_start' : answer_start,'answer_end' : answer_end})
    return new_answers

In [7]:
def prep_data(dataset):
    ans_start = dataset['ans_start'].tolist()
    text = dataset['text'].tolist()
    context = dataset['context'].tolist()
    questions = dataset['question'].tolist()
    contexts = dataset['context'].tolist()
    answers = add_end_idx(text, ans_start, context)
    return {
        'question': questions,
        'context': contexts,
        'answers': answers
    }

In [8]:
tr_dataset = prep_data(df_train)
val_dataset = prep_data(df_test)
tr_dataset['answers'][1]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

{'text': '۱۴', 'answer_start': 173, 'answer_end': 175}

In [9]:
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

Downloading:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

In [ ]:
new_context, new_question, new_answer = [[] for j in range(0,3)]
len_data = len(tr_dataset['answers'])
print('total size of data is {}'.format(len_data))
for i in range(0,len_data):
  tokenized_answer = tokenizer.tokenize(tr_dataset['answers'][i]['text'])
  tokenized_context = tokenizer.tokenize(tr_dataset['context'][i])
  tokenized_question = tokenizer.tokenize(tr_dataset['question'][i])
  num = len(tokenized_context) + len(tokenized_question)
  if num > (min_len - 3) and num <= (max_len - 3) : #3 for three special tokens, cls and 2 sep
    if '[UNK]' not in (tokenized_answer):
      
      new_context.append(tr_dataset['context'][i])
      new_question.append(tr_dataset['question'][i])
      new_answer.append({'text': tr_dataset['answers'][i]['text']
                         ,'answer_start' : tr_dataset['answers'][i]['answer_start'],
                         'answer_end' : tr_dataset['answers'][i]['answer_end']})
      
      

print('number of data without [UNK] and containing 128-256 tokens is {}'.format(len(new_context)))  
print('percentage of data without [UNK] and containing 128-256 tokens is is {}'.format(100 * len(new_context)/len_data))

new_tr_dataset = {
        'question': new_question,
        'context': new_context,
        'answers': new_answer
      }     

total size of data is 84713
number of data without [UNK] and containing 128-256 tokens is 68858
percentage of data without [UNK] and containing 128-256 tokens is is 81.2838643419546


In [ ]:
new_context, new_question, new_answer = [[] for j in range(0,3)]
len_data = len(val_dataset['answers'])
#print('total size of data is {}'.format(len_data))
for i in range(0,len_data):
  tokenized_answer = tokenizer.tokenize(val_dataset['answers'][i]['text'])
  tokenized_context = tokenizer.tokenize(val_dataset['context'][i])
  tokenized_question = tokenizer.tokenize(val_dataset['question'][i])
  num = len(tokenized_context) + len(tokenized_question)
  if num > (min_len - 3) and num <= (max_len - 3) : #3 for three special tokens, cls and 2 sep
    if '[UNK]' not in (tokenized_answer):
      
      new_context.append(val_dataset['context'][i])
      new_question.append(val_dataset['question'][i])
      new_answer.append({'text': val_dataset['answers'][i]['text']
                         ,'answer_start' : val_dataset['answers'][i]['answer_start'],
                         'answer_end' : val_dataset['answers'][i]['answer_end']})
      
      

#print('number of data without [UNK] and containing 128-256 tokens is {}'.format(len(new_context)))  
#print('percentage of data without [UNK] and containing 128-256 tokens is is {}'.format(100 * len(new_context)/len_data))

new_val_dataset = {
        'question': new_question,
        'context': new_context,
        'answers': new_answer
      }   

In [ ]:
validation = pd.DataFrame.from_dict(new_val_dataset)
train = pd.DataFrame.from_dict(new_tr_dataset)
train.head()

,question,context,answers
0,لیگ برتر انگلستان موفق به کسب چند عنوان قهرمان...,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,"{'text': '۱۴', 'answer_start': 173, 'answer_en..."
1,بیشترین بازی بدون باخت پیاپی متعلق به کدام باش...,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,"{'text': 'باشگاه فوتبال انگلیسی', 'answer_star..."
2,باشگاه فوتبال آرسنال موفق به کسب چند عنوان قهر...,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,"{'text': '۱۳', 'answer_start': 119, 'answer_en..."
3,باشگاه فوتبال آرسنال چند عنوان قهرمانی در جام ...,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,"{'text': '۱۶', 'answer_start': 214, 'answer_en..."
4,باشگاه فوتبال آرسنال چند قهرمانی در جام اتحادی...,باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...,"{'text': 'دو قهرمانی', 'answer_start': 249, 'a..."


In [ ]:
validation.head()

,question,context,answers
0,عده بسیاری از فیلم سازان به چه علت جذب تلویزیو...,از آغاز شکل‌گیری تلویزیون به علت امکانات مالی...,"{'text': 'امکانات مالی خوب', 'answer_start': 3..."
1,در دلائل الامامه چه نام‌هایی برای دختران محمد...,در تمامی منابع از هادی و موسی معروف به مُبرقع ...,"{'text': 'خدیجه، حکیمه و ام کلثوم', 'answer_st..."
2,کدام رویداد توجه جهانگیر امپراتور گورکانی را ب...,تاجران هر از چندی با همتایان هلندی، فرانسوی و ...,{'text': 'پیروزی آن‌ها بر پرتغالی‌ها در نبرد ...
3,استفاده از نرم‌افزار استیم‌پلن شبیه‌سازی و تحل...,این تیم توانسته است با به‌کارگیری سیستم ارزیاب...,"{'text': 'بی‌پی و آرکو', 'answer_start': 252,..."
4,سیاستمداران قرن نوزدهم آمریکا چه کسانی بودند؟,یورک این فرضیه را که نام آلبوم ستیزی برای انت...,"{'text': '', 'answer_start': -1, 'answer_end':..."


In [ ]:
print(len(train))
print(len(validation))

61972
6886


In [ ]:
def prep_data2(dataset):
    questions = dataset['question'].tolist()
    contexts = dataset['context'].tolist()
    answers = dataset['answers'].tolist()
    return {
        'question': questions,
        'context': contexts,
        'answers': answers
    }

tr_dataset = prep_data2(train)
val_dataset = prep_data2(validation)

In [ ]:
train_data = tokenizer(tr_dataset['context'], tr_dataset['question'],
                  truncation = False, padding='max_length',
                  max_length = max_len, return_tensors='pt')

validation_data = tokenizer(val_dataset['context'], val_dataset['question'],
                  truncation = False, padding='max_length',
                  max_length = max_len, return_tensors='pt')

In [ ]:
#modify [unk] token in answers

def char_idx_to_token_idx(tokenizer, char_idx, sentence):
  char = [0 if sentence[i] == ' ' or sentence[i] == '‌' else 1 for i in range(0,len(sentence))]
  tokens = tokenizer.tokenize(sentence)
  index = char_idx
  for i in range(0,index):
    if char[i] != 1 :
      index += -1
  counter = 0
  flag_continue = True
  token_index = 0
  for i in range(len(tokens)):
    if tokens[i].startswith('##'): 
      tokens[i] = tokens[i][2:]
    for j in range(0,len(tokens[i])):
      counter += 1
      if counter == index :
        flag_continue == False
        token_index = i
        break
    if flag_continue == False :
      break    
  return token_index             

def add_token_positions(tokenizer, encodings, answers, contexts):
    target_start_list = []
    target_end_list = []
    for i in tqdm(range(len(answers))):
        target_start =   [0] * max_len
        target_end =   [0] * max_len
        start_idx = answers[i]['answer_start']
        end_idx = answers[i]['answer_end']
        if start_idx <= len(contexts[i]) and end_idx <= len(contexts[i]):
          #answerable question
          if start_idx != -1 & end_idx != -1:
            start_token_idx = char_idx_to_token_idx(tokenizer, start_idx, contexts[i]) + 1
            end_token_idx = char_idx_to_token_idx(tokenizer, end_idx, contexts[i]) + 1
            target_start[start_token_idx] = 1
            target_end[end_token_idx] = 1
            target_start_list.append(target_start)
            target_end_list.append(target_end)
    
          #unanswerable question
          else:
            target_start[0] = 1
            target_end[0] = 1
            target_start_list.append(target_start)
            target_end_list.append(target_end)
        else: 
          continue    
    encodings.update({'targets_start' : target_start_list, 'targets_end' : target_end_list})

add_token_positions(tokenizer, train_data, tr_dataset['answers'], tr_dataset['context'])
add_token_positions(tokenizer, validation_data, val_dataset['answers'], val_dataset['context'])

  0%|          | 0/61972 [00:00<?, ?it/s]

  0%|          | 0/6886 [00:00<?, ?it/s]

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data

    def __getitem__(self, idx):
                
        return {key: torch.tensor(val[idx]) for key, val in self.data.items()}
         
    def __len__(self):
        return len(self.data.input_ids)

In [ ]:
train_dataset = CustomDataset(train_data)
print(len(train_dataset))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True)

validation_dataset = CustomDataset(validation_data)
print(len(validation_dataset))
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size = 32, shuffle = False)

61972
6886


In [ ]:
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'

class QAModel(nn.Module):

    def __init__(self):
        super(QAModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH, return_dict=False)
        self.classifier = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        sequence_output, pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)
        
         # (batch_size, num_tokens, 768)
        logits = self.classifier(sequence_output)
        # (batch_size, num_tokens, 2)
        # (batch_size, num_tokens, 1), (batch_size, num_tokens, 1)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        # (batch_size, num_tokens), (batch_size, num_tokens)

        return start_logits, end_logits

In [ ]:
def loss_fn(out1,out2,target1,target2):
  l1 = nn.BCEWithLogitsLoss()(out1, target1.float())
  l2 = nn.BCEWithLogitsLoss()(out2, target2.float())
  return l1 + l2

In [ ]:
def evaluate_f1(start_pred, start_target, end_pred, end_target):
  
  pred = np.arange(start_pred,end_pred + 1)
  tar = np.arange(start_target,end_target + 1)

  #number of tokens shared between prediction and target
  tp_list = list(set.intersection(*map(set, [pred, tar])))
  #number of tokens in prediction not in target
  fp_list = list(set(pred).symmetric_difference(set(tp_list)))
  #number of tokens in target not in prediction
  fn_list = list(set(tar).symmetric_difference(set(tp_list)))

  tp, fp, fn = len(tp_list), len(fp_list), len(fn_list)

  if (tp + fp) != 0 :
    precision = tp/(tp + fp)
  else :
    precision = 0  

  if (tp + fn) != 0 :
    recall = tp/(tp + fn)
  else :
    recall = 0    

  if (precision + recall) != 0 :
    f1 = (2 * precision * recall) / (precision + recall)     
  else : 
    f1 = 0   

  return f1 


def generate_indexes(start_logits, end_logits, N, max_index_list):
  
  output_start = start_logits
  output_end = end_logits
  dimension = (output_start.shape[1])
  #first [SEP] token index, we want to choose the span within the user utterance

  list_start, list_end = [], []
  for n in range(0,output_start.shape[0]):
    start_indexes = np.arange(output_start.shape[1])
    start_probs = output_start[n]
    list_start.append(dict(zip(start_indexes, start_probs)))

    end_indexes = np.arange(output_start.shape[1])
    end_probs = output_end[n]
    list_end.append(dict(zip(end_indexes, end_probs)))

  sorted_start_list, sorted_end_list = [], []
  for j in range(0,len(list_start)):
    sort_start_probs = sorted(list_start[j].items(), key=lambda x: x[1], reverse=True)
    sort_end_probs = sorted(list_end[j].items(), key=lambda x: x[1], reverse=True)
    sorted_start_list.append(sort_start_probs)
    sorted_end_list.append(sort_end_probs)
    
  final_start_idx, final_end_idx = [], []

  for c in range(0,len(list_start)):
    start_idx, end_idx, prob = 0, 0, 0
    for a in range(0,N):
      for b in range(0,N):
        if (sorted_start_list[c][a][1] + sorted_end_list[c][b][1]) > prob :
          if (sorted_start_list[c][a][0] <= sorted_end_list[c][b][0]) and (sorted_end_list[c][b][0] < max_index_list[c]) :
            prob = sorted_start_list[c][a][1] + sorted_end_list[c][b][1]
            start_idx = sorted_start_list[c][a][0]
            end_idx = sorted_end_list[c][b][0]
    final_start_idx.append(start_idx)    
    final_end_idx.append(end_idx)      

  return final_start_idx, final_end_idx
  

def evaluate_model(start_logits, end_logits, N, max_index_list, target_start, target_end):
  final_start_idx, final_end_idx = generate_indexes(start_logits, end_logits, N, max_index_list)
  f1 = []
  for i in range(0,len(final_start_idx)):
    f1.append(evaluate_f1(final_start_idx[i], target_start[i], final_end_idx[i], target_end[i]))
  

  return np.mean(f1)

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = QAModel()
model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

cuda:0


Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation

Evaluate Model on Validation Dataset Before any Training

In [ ]:
softmax = torch.nn.Softmax(dim=1)
model.eval()
l = 0
n_validation_batches = len(validation_loader)
exact_match,f1 = [],[]

with torch.no_grad():  
  loop2 = tqdm(validation_loader)
  for content in loop2:

    input_ids = content['input_ids'].to(device)
    temp_ids = input_ids.cpu().data.numpy().tolist()
    max_ind = [temp_ids[i].index(4) for i in range(0,len(temp_ids))] #index of first [sep]
    attention_mask = content['attention_mask'].to(device)
    token_type_ids = content['token_type_ids'].to(device)
    y_start = content['targets_start'].to(device)
    y_end = content['targets_end'].to(device)
    out_start, out_end = model(input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
    start_pred = softmax(out_start)
    end_pred = softmax(out_end)
    loss2 = loss_fn(out_start, out_end, y_start, y_end)
    l += loss2
    start_label = np.nonzero(y_start).cpu().data.numpy()
    end_label = np.nonzero(y_end).cpu().data.numpy()
    tensor_label = torch.stack((torch.tensor(start_label[:,1]), torch.tensor(end_label[:,1])), -1)
    start_log, end_log = generate_indexes(start_pred, end_pred, N, max_ind)
    start_log = np.array(start_log)
    end_log = np.array(end_log)
    tensor_pred = torch.stack((torch.tensor(start_log), torch.tensor(end_log)), -1)
    ex_ma = sum([1 if (tensor_pred[i][0] == tensor_label[i][0] and tensor_pred[i][1] == tensor_label[i][1]) else 0 for i in range(0,len(tensor_pred))])/len(tensor_pred)
    exact_match.append(ex_ma)
    f1.append(evaluate_model(start_pred.cpu().data.numpy(), end_pred.cpu().data.numpy()
    , N, max_ind, start_label[:,1], end_label[:,1]))

  valid_loss = l / n_validation_batches
  print('Test Loss: {:.4f}, exact_match: {:.2f}, F1: {:.2f}'.format(
                                                                      valid_loss,
                                                                      100 * np.mean(exact_match),
                                                                      100 * np.mean(f1)))

  0%|          | 0/216 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Test Loss: 1.1515, exact_match: 1.58, F1: 6.55


In [ ]:
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
from transformers import PreTrainedModel

class QAModel2(PreTrainedModel):

    def __init__(self, config):
        super(QAModel2, self).__init__(config)
        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH, return_dict=False)
        #self.dropout = nn.Dropout(p = dropout_rate)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        sequence_output, pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)
        
        #we apply dropout to the sequence output, tensor has shape (batch_size, sequence_length, 768)
        #sequence_output = self.dropout(sequence_output)
        # (batch_size, num_tokens, 768)
        logits = self.classifier(sequence_output)
        # (batch_size, num_tokens, 2)
        # (batch_size, num_tokens, 1), (batch_size, num_tokens, 1)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        # (batch_size, num_tokens), (batch_size, num_tokens)

        return start_logits, end_logits

In [ ]:
from transformers import AutoConfig, AutoTokenizer
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = QAModel2(config)
model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected i

QAModel2(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [ ]:
n_epochs = 10
n_train_batches = len(train_loader)
n_validation_batches = len(validation_loader)
softmax = torch.nn.Softmax(dim=1)
best_valid_loss = float('inf')

for epoch in range(n_epochs):
  
    loop = tqdm(train_loader)
    train_running_loss, validation_running_loss = 0.0, 0.0
    exact_match = []
    f1 = []
    for batch in loop:
       
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        y_start = batch['targets_start'].to(device)
        y_end = batch['targets_end'].to(device)
        out_start, out_end = model(input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)  
        loss = loss_fn(out_start, out_end, y_start, y_end)
        train_running_loss += loss
        loss.backward()
        optim.step()
        loop.set_description(f'Epoch {epoch+1} - Training  ')

    with torch.no_grad():  
        loop2 = tqdm(validation_loader)
        for content in loop2:
            
            input_ids = content['input_ids'].to(device)
            temp_ids = input_ids.cpu().data.numpy().tolist()
            max_ind = [temp_ids[i].index(4) for i in range(0,len(temp_ids))] #index of first [sep]
            attention_mask = content['attention_mask'].to(device)
            token_type_ids = content['token_type_ids'].to(device)
            y_start = content['targets_start'].to(device)
            y_end = content['targets_end'].to(device)
            out_start, out_end = model(input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
            start_pred = softmax(out_start)
            end_pred = softmax(out_end)
            loss2 = loss_fn(out_start, out_end, y_start, y_end)
            validation_running_loss += loss2
            start_label = np.nonzero(y_start).cpu().data.numpy()
            end_label = np.nonzero(y_end).cpu().data.numpy()
            tensor_label = torch.stack((torch.tensor(start_label[:,1]), torch.tensor(end_label[:,1])), -1)
            start_log, end_log = generate_indexes(start_pred, end_pred, N, max_ind)
            start_log = np.array(start_log)
            end_log = np.array(end_log)
            tensor_pred = torch.stack((torch.tensor(start_log), torch.tensor(end_log)), -1)
            ex_ma = sum([1 if (tensor_pred[i][0] == tensor_label[i][0] and tensor_pred[i][1] == tensor_label[i][1]) else 0 for i in range(0,len(tensor_pred))])/len(tensor_pred)
            exact_match.append(ex_ma)
            f1.append(evaluate_model(start_pred.cpu().data.numpy(), end_pred.cpu().data.numpy()
            , N, max_ind, start_label[:,1], end_label[:,1]))
            loop2.set_description(f'Epoch {epoch+1} - Validation')

    valid_loss = validation_running_loss / n_validation_batches
    if valid_loss < best_valid_loss:
      best_valid_loss = valid_loss
      torch.save(model.state_dict(),'/content/gdrive/MyDrive/saved_weights_dropout_0.05.pt')
      print('New Model Weights Saved in Drive!')
    print('Epoch {}: Train Loss: {:.5f}, Test Loss: {:.5f}, exact_match: {:.2f}%, F1: {:.2f}%'.format(epoch + 1,
                                                                               train_running_loss / n_train_batches,
                                                                               valid_loss,
                                                                               100 * np.mean(exact_match),
                                                                               100 * np.mean(f1)))

  0%|          | 0/1937 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


  0%|          | 0/216 [00:00<?, ?it/s]

New Model Weights Saved in Drive!
Epoch 1: Train Loss: 0.0238, Test Loss: 0.0174, exact_match: 38.68%, F1: 73.02%


  0%|          | 0/1937 [00:00<?, ?it/s]

...

  0%|          | 0/1937 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

Epoch 4: Train Loss: 0.0090, Test Loss: 0.0168, exact_match: 45.43%, F1: 75.74%


  0%|          | 0/1937 [00:00<?, ?it/s]

KeyboardInterrupt: ignored